In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchviz import make_dot

# 定义一个多项式函数
class PolynomialModel(nn.Module):
    def __init__(self, degree):
        super(PolynomialModel, self).__init__()
        self.degree = degree
        self.layers = nn.ModuleList([nn.Linear(1, 1, bias=False) for _ in range(degree + 1)])

    def forward(self, x):
        # 计算多项式
        out = torch.zeros_like(x)
        for i, layer in enumerate(self.layers):
            out += layer(x) * x**i
        return out

# 初始化模型
degree = 3  # 多项式的最高次数
model = PolynomialModel(degree)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 生成一些随机数据来模拟x和y的关系
x_data = torch.randn(100, 1)  # 生成100个随机样本
y_data = torch.cat([x_data**i for i in range(degree + 1)], 1).sum(1, keepdim=True)  # 计算y的理论值

# 添加随机噪声来模拟真实数据
y_data += torch.randn_like(y_data) * 0.1  # 假设噪声是均值为0，标准差为0.1的正态分布

# 训练模型
for epoch in range(1000):
    optimizer.zero_grad()
    # 生成随时间变化的随机变量 τ(t)
    tau = torch.randn_like(x_data)  # 假设 x_data 已经被定义，并且具有正确的时间维度
    
    # 计算模型的输出
    y_pred = model(x_data) + tau  # 将随机变量添加到模型的输出中
    loss = criterion(y_pred, y_data)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# 可视化模型
x_test = torch.linspace(-3, 3, 100).unsqueeze(1)  # 创建一个测试集
y_pred = model(x_test)

plt.scatter(x_data.numpy(), y_data.numpy(), color='blue', label='Data Points')
plt.plot(x_test.numpy(), y_pred.detach().numpy(), color='red', label='Polynomial Fit')
plt.legend()
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义一个多项式函数
class PolynomialModel(nn.Module):
    def __init__(self, degree):
        super(PolynomialModel, self).__init__()
        self.degree = degree
        self.layers = nn.ModuleList([nn.Linear(1, 1, bias=False) for _ in range(degree + 1)])

    def forward(self, x):
        # 计算多项式
        out = torch.zeros_like(x)
        for i, layer in enumerate(self.layers):
            out += layer(x) * x**i
        return out

# 初始化模型
degree = 3  # 多项式的最高次数
model = PolynomialModel(degree)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练模型
num_epochs = 1000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # 生成随时间变化的随机变量 τ(t)
    tau = torch.randn_like(x_data)  # 假设 x_data 已经被定义，并且具有正确的时间维度
    
    # 计算模型的输出
    y_pred = model(x_data) + tau  # 将随机变量添加到模型的输出中
    
    # 计算损失
    loss = criterion(y_pred, y_data)
    
    # 反向传播和优化
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from power_plant_dataset import PowerPlantDataset
from torch.utils.data import Dataset, DataLoader
dataset = PowerPlantDataset(csv_file='fdcl/split/df_hdrm.csv')
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)

rou_train = 1.2
rou_test = 1.3

# 定义一个简单的多项式函数
class PolynomialRegressor(nn.Module):
    def __init__(self, degree):
        super(PolynomialRegressor, self).__init__()
        self.degree = degree
        self.coefficients = nn.Parameter(torch.randn(degree + 1))  # 随机初始化系数

    def forward(self, x, rou):
        # 计算多项式的值
        out = torch.sum(self.coefficients * x * rou, dim=1)
        return out

In [2]:
# 初始化模型
degree = 3  # 多项式的最高次数
model = PolynomialRegressor(degree)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


# 训练模型
for epoch in range(10):
    optimizer.zero_grad()
    for batch_features, batch_labels in dataloader:
        y_pred = model(-0.5, rou_train)  # 使用训练集的随机变量参数
        loss = criterion(y_pred, batch_labels)
        loss.backward()
        optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# 使用测试集测试模型
with torch.no_grad():
    for batch_features, batch_labels in dataloader:
        y_pred = model(batch_features, rou_test)  # 使用测试集的随机变量参数
        print(f'Test Loss: {criterion(y_pred, batch_labels).item()}')

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)